In [25]:
from autogen_agentchat.agents import AssistantAgent, UserProxyAgent
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.ui import Console
from dotenv import load_dotenv
import asyncio
import os

load_dotenv()
api_key = os.getenv("GOOGLE_API_KEY")
open_router_key = os.getenv("OPENROUTER_API_KEY")

In [26]:
model_client = OpenAIChatCompletionClient(
    model='gemini-2.0-flash',
    api_key=api_key,
)

In [27]:
from autogen_core.memory import ListMemory, MemoryContent, MemoryMimeType

user_memory = ListMemory()

await user_memory.add(MemoryContent(content="The Weather Should be in metric units", mime_type=MemoryMimeType.TEXT))

await user_memory.add(MemoryContent(content="Meal Recipe must be Vegan", mime_type=MemoryMimeType.TEXT))

In [28]:
async def get_weather(city: str, units: str = "imperial") -> str:
    if units == "imperial":
        return f"The weather in {city} is 73 °F and Sunny."
    elif units == "metric":
        return f"The weather in {city} is 23 °C and Sunny."
    else:
        return f"Sorry, I don't know the weather in {city}."

In [29]:
assistant_agent = AssistantAgent(
    name="assistant_agent",
    model_client=model_client,
    tools=[get_weather],
    memory=[user_memory],
    system_message=None
)

In [30]:
stream = assistant_agent.run_stream(task="What is the weather in New York?")
await Console(stream)

---------- TextMessage (user) ----------
What is the weather in New York?
---------- MemoryQueryEvent (assistant_agent) ----------
[MemoryContent(content='The Weather Should be in metric units', mime_type=<MemoryMimeType.TEXT: 'text/plain'>, metadata=None), MemoryContent(content='Meal Recipe must be Vegan', mime_type=<MemoryMimeType.TEXT: 'text/plain'>, metadata=None)]
---------- ToolCallRequestEvent (assistant_agent) ----------
[FunctionCall(id='', arguments='{"city":"New York","units":"metric"}', name='get_weather')]
---------- ToolCallExecutionEvent (assistant_agent) ----------
[FunctionExecutionResult(content='The weather in New York is 23 °C and Sunny.', name='get_weather', call_id='', is_error=False)]
---------- ToolCallSummaryMessage (assistant_agent) ----------
The weather in New York is 23 °C and Sunny.


TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, content='What is the weather in New York?', type='TextMessage'), MemoryQueryEvent(source='assistant_agent', models_usage=None, metadata={}, content=[MemoryContent(content='The Weather Should be in metric units', mime_type=<MemoryMimeType.TEXT: 'text/plain'>, metadata=None), MemoryContent(content='Meal Recipe must be Vegan', mime_type=<MemoryMimeType.TEXT: 'text/plain'>, metadata=None)], type='MemoryQueryEvent'), ToolCallRequestEvent(source='assistant_agent', models_usage=RequestUsage(prompt_tokens=46, completion_tokens=8), metadata={}, content=[FunctionCall(id='', arguments='{"city":"New York","units":"metric"}', name='get_weather')], type='ToolCallRequestEvent'), ToolCallExecutionEvent(source='assistant_agent', models_usage=None, metadata={}, content=[FunctionExecutionResult(content='The weather in New York is 23 °C and Sunny.', name='get_weather', call_id='', is_error=False)], type='ToolCallExecutionEvent

In [31]:
await assistant_agent.model_context.get_messages()

[UserMessage(content='What is the weather in New York?', source='user', type='UserMessage'),
 SystemMessage(content='\nRelevant memory content (in chronological order):\n1. The Weather Should be in metric units\n2. Meal Recipe must be Vegan\n', type='SystemMessage'),
 AssistantMessage(content=[FunctionCall(id='', arguments='{"city":"New York","units":"metric"}', name='get_weather')], thought=None, source='assistant_agent', type='AssistantMessage'),
 FunctionExecutionResultMessage(content=[FunctionExecutionResult(content='The weather in New York is 23 °C and Sunny.', name='get_weather', call_id='', is_error=False)], type='FunctionExecutionResultMessage')]

In [39]:
assistant_agent = AssistantAgent(
    name="assistant_agent",
    model_client=model_client,
    tools=[get_weather],
    memory=[user_memory],
    system_message=None
)

In [40]:
stream = assistant_agent.run_stream(task="Write brief meal recipe with ingredients and instructions for a vegan meal.")
await Console(stream)

---------- TextMessage (user) ----------
Write brief meal recipe with ingredients and instructions for a vegan meal.
---------- MemoryQueryEvent (assistant_agent) ----------
[MemoryContent(content='The Weather Should be in metric units', mime_type=<MemoryMimeType.TEXT: 'text/plain'>, metadata=None), MemoryContent(content='Meal Recipe must be Vegan', mime_type=<MemoryMimeType.TEXT: 'text/plain'>, metadata=None)]
---------- TextMessage (assistant_agent) ----------
Okay, I will provide you with a vegan meal recipe. How about a lentil soup?

Ingredients:

*   1 cup red lentils
*   1 onion, chopped
*   2 carrots, chopped
*   2 cloves garlic, minced
*   4 cups vegetable broth
*   1 teaspoon cumin
*   1/2 teaspoon turmeric
*   Salt and pepper to taste
*   1 tablespoon olive oil

Instructions:

1.  Heat olive oil in a pot over medium heat. Add onion and carrots and cook until softened, about 5 minutes.
2.  Add garlic, cumin, and turmeric and cook for another minute until fragrant.
3.  Add lent

TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, content='Write brief meal recipe with ingredients and instructions for a vegan meal.', type='TextMessage'), MemoryQueryEvent(source='assistant_agent', models_usage=None, metadata={}, content=[MemoryContent(content='The Weather Should be in metric units', mime_type=<MemoryMimeType.TEXT: 'text/plain'>, metadata=None), MemoryContent(content='Meal Recipe must be Vegan', mime_type=<MemoryMimeType.TEXT: 'text/plain'>, metadata=None)], type='MemoryQueryEvent'), TextMessage(source='assistant_agent', models_usage=RequestUsage(prompt_tokens=51, completion_tokens=200), metadata={}, content='Okay, I will provide you with a vegan meal recipe. How about a lentil soup?\n\nIngredients:\n\n*   1 cup red lentils\n*   1 onion, chopped\n*   2 carrots, chopped\n*   2 cloves garlic, minced\n*   4 cups vegetable broth\n*   1 teaspoon cumin\n*   1/2 teaspoon turmeric\n*   Salt and pepper to taste\n*   1 tablespoon olive oil\n\nIns